In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver

from bs4 import BeautifulSoup

from selenium.webdriver.support.ui import Select
import time
import numpy as np
import pandas as pd

def crawler(ticker, sd, ed):
    sdList = sd.split('-')
    
    edList = ed.split('-')
    
    sdList[0] = int(sdList[0]) - 1911
    
    edList[0] = int(edList[0]) - 1911
    
    sdList[0], edList[0] = str(sdList[0]), str(edList[0])
    
    sd = ''.join(sdList)
    ed = ''.join(edList)
    
    dateList = []
    chrome_options = Options()
    chrome_options.add_argument("--headless")       # define headless

    # add the option when creating driver
    driver = webdriver.Chrome(chrome_options=chrome_options)
#     driver = webdriver.Chrome()

    driver.get("https://mops.twse.com.tw/mops/web/STAMAK03_1")
    # time.sleep(1)
    driver.find_element_by_id("co_id").click()
    driver.find_element_by_id("co_id").clear()
    driver.find_element_by_id("co_id").send_keys(ticker)
    driver.find_element_by_id("b_date").click()
    driver.find_element_by_id("b_date").clear()
    driver.find_element_by_id("b_date").send_keys(sd)
    driver.find_element_by_id("e_date").click()
    driver.find_element_by_id("e_date").clear()
    driver.find_element_by_id("e_date").send_keys(ed)
    time.sleep(0.5)
    driver.find_element_by_id("nav02").click()
    driver.find_element_by_xpath(u"//input[@value=' 查詢 ']").click()
    time.sleep(3)
    driver.find_element_by_xpath(u"//img[@alt='開新視窗']").click()

    html = driver.page_source

    bs4 = BeautifulSoup(html)

    table = bs4.find_all('table', {'class':'hasBorder'})
    if len(table) == 0:
        print('There are no data.')
        return dateList, []
    table = table[0].find_all('td')
    tmpList = list()
    dataNP = []
    for i, t in enumerate(table):
        tmpList.append(t.text)
        if i % 11 == 10 and i == 10:
            dataNP = tmpList
            tmpList = list()
        elif i % 11 == 10:
            tmpList = np.array(tmpList)[None,:]
            dataNP = np.vstack((dataNP, tmpList))
            tmpList = list()
    df = pd.DataFrame(dataNP)
    colName = ['公司代號',	'公司名稱',	'設質人身份別',	'設質人姓名',	'質設異動發生日期',	'設質股數',	'解質股數',	'累積質設股數',	'質權人姓名',	'備註',	'申報日期']
    df.columns = colName
    df = df.drop(columns=['公司代號', '公司名稱', '備註'])
#     display(df)
#     print(df.loc[df['解質股數'] !=0])
    return dateList, df

In [ ]:
a = input('aa')